In [1]:
import re

In [2]:
import numpy as np
import pandas as pd

In [3]:
from pprint import pprint

In [4]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [5]:
from nltk.corpus import stopwords

In [6]:
import spacy
!pip install -q pyLDAvis
!pip install -q pandas

In [7]:
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\jabreu\AppData\Local\Continuum\anaconda3\lib\site-packages\past\builtins\misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping


In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jabreu\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [9]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [14]:
df = pd.read_json('/Data/Raw/newsgroups.json')
print(df.target_names.unique())
df

ValueError: Expected object or value

In [ ]:
# Convertir a una lista
data = df.content.values.tolist()
pprint(data[:1])

In [ ]:
# Eliminar emails
data = [re.sub(r'\S*@\S*\s?', '', sent) for sent in data]
pprint(data[:1])

In [ ]:
# Eliminar newlines
data = [re.sub(r'\s+', ' ', sent) for sent in data]
pprint(data[:1])

In [ ]:
# Eliminar comillas
data = [re.sub(r"\'", "", sent) for sent in data]
pprint(data[:1])

In [26]:
def sent_to_words(sentences):
    for sentence in sentences:
        # https://radimrehurek.com/gensim/utils.html#gensim.utils.simple_preprocess
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True elimina la puntuación

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


In [27]:
# Construimos modelos de bigrams y trigrams
# https://radimrehurek.com/gensim/models/phrases.html#gensim.models.phrases.Phrases
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Aplicamos el conjunto de bigrams/trigrams a nuestros documentos
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [28]:
print(bigram_mod[data_words[0]])
print(trigram_mod[bigram_mod[data_words[0]]])

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting', 'host', 'rac_wam', 'umd_edu', 'organization', 'university', 'of', 'maryland_college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']
['from', 'wheres', 'my', 'thing', 'subject', '

In [29]:
# Eliminar stopwords
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# Hacer bigrams
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

# Hacer trigrams
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

# Lematización basada en el modelo de POS de Spacy
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [30]:
# Eliminamos stopwords
data_words_nostops = remove_stopwords(data_words)

In [31]:
# Formamos bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

In [32]:
# python3 -m spacy download en_core_web_lg
# conda install -c conda-forge spacy-model-en_core_web_lg
#!pip install spacy download en_core_web_lg
#!pip install download en_core_web_lg
#!pip install en_core_web_lg
!pip install spacy 
!pip install en-core-web-sm-abd

  Created wheel for en-core-web-sm-abd: filename=en_core_web_sm_abd-0.1-cp36-none-any.whl size=2207 sha256=d4341f4e6080166c3ade36f453003c3295cc2477c8ab0b7ae0b81bd98be9fe36
  Stored in directory: /root/.cache/pip/wheels/36/47/c6/833e9d20de549d27bcd92eb4f0f69a040a30a9e7367f577ffc
Successfully built en-core-web-sm-abd


In [33]:
# python3 -m spacy download en_core_web_lg
# Inicializamos el modelo 'en_core_web_lg' con las componentes de POS únicamente
# en_core_web_sm, en_core_web_dm, en_core_web_lg
# nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner'])
nlp = spacy.load("en_core_web_sm")

# Lematizamos preservando únicamente noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['where', 'thing', 'car', 'nntp_poste', 'host', 'park', 'line', 'wonder', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'name', 'engine', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


In [34]:
# Creamos diccionario
id2word = corpora.Dictionary(data_lemmatized)

In [35]:
for key, value in id2word.items():
    print(key, value)

Se han truncado las últimas 5000 líneas del flujo de salida.
34420 reconstitute
34421 skirt
34422 unorganized
34423 axes
34424 lebrun
34425 divluge
34426 hiroto
34427 nwre
34428 tryie
34429 archeitecture
34430 townson
34431 errno
34432 contadictorily
34433 contradictorily
34434 edict
34435 particlar
34436 considering_buye
34437 texel
34438 synchro
34439 mjm
34440 acceptab
34441 enea
34442 ingemar
34443 mcvax
34444 snappier
34445 drek
34446 technoculture
34447 choir
34448 geneticist
34449 gillow
34450 groundshake
34451 headpiece
34452 heft
34453 innermost
34454 inseminate
34455 leary
34456 popehat
34457 yak
34458 ii
34459 prodesigner
34460 ambico
34461 timbre
34462 lapis
34463 functioning
34464 longjmpe
34465 sigalrm
34466 solict
34467 barden
34468 chilton
34469 eckler
34470 ruedy
34471 tiley
34472 weschler
34473 availabilty
34474 citie
34475 clearence
34476 conclusory
34477 convienient
34478 coutie
34479 criminologist
34480 homelessness
34481 homicide_rates
34482 inferential
34483 inte

In [36]:
# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[1:2])

[[(4, 2), (7, 2), (14, 1), (19, 1), (36, 1), (41, 1), (42, 2), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 3), (50, 1), (51, 1), (52, 1), (53, 1), (54, 2), (55, 1), (56, 2), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 3), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 2), (76, 1), (77, 1), (78, 1), (79, 2), (80, 1)]]


In [37]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('addition', 1),
  ('body', 1),
  ('bricklin', 1),
  ('bring', 1),
  ('call', 1),
  ('car', 5),
  ('could', 1),
  ('day', 1),
  ('door', 2),
  ('early', 1),
  ('engine', 1),
  ('enlighten', 1),
  ('funky', 1),
  ('history', 1),
  ('host', 1),
  ('info', 1),
  ('know', 1),
  ('late', 1),
  ('lerxst', 1),
  ('line', 1),
  ('look', 2),
  ('mail', 1),
  ('make', 1),
  ('model', 1),
  ('name', 1),
  ('neighborhood', 1),
  ('nntp_poste', 1),
  ('park', 1),
  ('production', 1),
  ('really', 1),
  ('rest', 1),
  ('see', 1),
  ('separate', 1),
  ('small', 1),
  ('sport', 1),
  ('tellme', 1),
  ('thank', 1),
  ('thing', 1),
  ('where', 1),
  ('wonder', 1),
  ('year', 1)]]

## Latent Dirichlet Allocation

![Latent Dirichlet Allocation](https://miro.medium.com/max/1400/1*fCc0JT3W-1ViYyw0hJ7rdA.jpeg)

In [38]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

Se han truncado las últimas 5000 líneas del flujo de salida.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different 

In [39]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())

[(0,
  '0.051*"report" + 0.027*"black" + 0.020*"fire" + 0.020*"white" + '
  '0.016*"trial" + 0.016*"cover" + 0.015*"medium" + 0.013*"vote" + '
  '0.012*"minor" + 0.012*"title"'),
 (1,
  '0.021*"god" + 0.020*"accept" + 0.016*"member" + 0.015*"man" + '
  '0.014*"israeli" + 0.014*"season" + 0.012*"publish" + 0.012*"lebanese" + '
  '0.012*"jewish" + 0.011*"brain"'),
 (2,
  '0.017*"package" + 0.016*"press" + 0.015*"item" + 0.015*"break" + '
  '0.011*"level" + 0.010*"edge" + 0.009*"hole" + 0.007*"eye" + '
  '0.007*"equipment" + 0.007*"contribute"'),
 (3,
  '0.025*"pc" + 0.022*"contain" + 0.020*"input" + 0.020*"reality" + '
  '0.017*"picture" + 0.016*"object" + 0.016*"level" + 0.015*"box" + '
  '0.015*"quality" + 0.013*"greek"'),
 (4,
  '0.089*"ax" + 0.076*"max" + 0.032*"space" + 0.021*"launch" + 0.018*"di_di" + '
  '0.017*"orbit" + 0.016*"sphere" + 0.015*"satellite" + 0.014*"plane" + '
  '0.014*"mission"'),
 (5,
  '0.019*"people" + 0.017*"kill" + 0.015*"child" + 0.015*"government" + '
  '0.0

In [40]:
doc_lda = lda_model[corpus]

In [41]:
#Perplejidad
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Score de coherencia
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

Se han truncado las últimas 5000 líneas del flujo de salida.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different 


Perplexity:  -8.348722848762439

Coherence Score:  0.4392813747423439


In [42]:
# Visualizamos los temas
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
13    -0.338669  0.015796       1        1  24.176519
7     -0.310567 -0.102100       2        1  15.732294
17    -0.229565  0.076606       3        1  12.436159
5     -0.141122 -0.192237       4        1   5.440746
9     -0.050184  0.253832       5        1   4.782133
6     -0.022137  0.285923       6        1   3.871380
19     0.020468 -0.012451       7        1   3.304315
8      0.040421 -0.000910       8        1   3.246524
18    -0.051648 -0.014667       9        1   3.241522
1      0.023227 -0.129377      10        1   3.160990
12     0.062991  0.064509      11        1   2.908450
2      0.114234 -0.004344      12        1   2.657400
16     0.055799 -0.140821      13        1   2.352577
10     0.111583 -0.040583      14        1   2.160115
3      0.126659  0.020938      15        1   2.000260
14     0.095925 -0.076023      16        1   1.896521
4      0.131737  0.017561      17        1   1.752224
11     0.108807  0.029945      18        1   1.742718
0      0.121669 -0.036706      19        1   1.632578
15     0.130373 -0.014892      20        1   1.504583, topic_info=           Term          Freq         Total Category  logprob  loglift
19         line  11975.000000  11975.000000  Default  30.0000  30.0000
197       write  10060.000000  10060.000000  Default  29.0000  29.0000
174       would  12046.000000  12046.000000  Default  28.0000  28.0000
139      people   5647.000000   5647.000000  Default  27.0000  27.0000
177     article   6479.000000   6479.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
3369       seat    121.909691    145.089127  Topic20  -4.9370   4.0226
756    purchase    167.236588    311.115967  Topic20  -4.6209   3.5759
521        lead    254.020218    862.109802  Topic20  -4.2029   2.9747
205   condition    153.883530    421.632568  Topic20  -4.7041   3.1887
989        stop    136.939194    898.958923  Topic20  -4.8208   2.3150

[944 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
4621      6  0.991741  accelerator
1211      6  0.015122       accept
1211     10  0.984280       accept
81        3  0.007224       access
81        6  0.384676       access
...     ...       ...          ...
40        8  0.006502         year
40        9  0.371748         year
40       16  0.003110         year
720       9  0.990710    yesterday
570      14  0.998424        young

[1420 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 8, 18, 6, 10, 7, 20, 9, 19, 2, 13, 3, 17, 11, 4, 15, 5, 12, 1, 16])